In [ ]:
import aocd
import dataclasses
import numpy as np
import enum

real_data = aocd.get_data(day=12, year=2022)
test_data = """Sabqponm
abcryxxl
accszExk
acctuvwj
abdefghi"""

In [ ]:
from typing import Sequence, Union, Type, Tuple
import itertools

def str_to_map(string) -> np.ndarray:
    """Converts a block string to 2d np.ndarray.
    
    Args:
        string: the input string that represents the heat map.
        
    Returns:
        heatmap, start_position, end_position
    """
    
    lines = []
    start_position = end_position = None
    for ii, line in enumerate(string.splitlines()):
        row = []
        for jj, char in enumerate(line):
            if char == "S":
                row.append(0)
                start_position = (ii, jj)
            elif char == "E":
                row.append(25)
                end_position = (ii, jj)
            else:
                row.append(ord(char) - 97)
        lines.append(np.array(row))
    return np.array(lines), start_position, end_position
        
@dataclasses.dataclass
class Node:
    """Defines a node instance.
    
    Args:
        parent: the parent node of this node. 
        pos: a tuple of this node's position.
    """
    pos: Tuple[int, int]
    parent: Union[None, Type["Node"]]
        
    def __post_init__(self):
        self.f = 0
        self.g = 0
        self.h = 0
        
    def __eq__(self, compared: Type["Node"]):
        return self.pos == compared.pos
    
    def get_path(self):
        """Get the entire path for this node.
        
        Returns:
            The path positions: Sequence[Tuple[int, int]]
        """
        path = []
        this_node = self
        while this_node.parent is not None:
            path.append(this_node.pos)
            this_node = this_node.parent
        return path[::-1]
        
    
def search(
    heatmap: np.ndarray, 
    start_pos: Tuple[int, int], 
    end_pos: Tuple[int, int]
) -> Sequence[Tuple[int, int]]:
    """Uses A* to search.
    
    Args:
        the_map: the heatmap of terrain.
        start_pos: the starting position.
        end_pos: the end position.
    
    Return:
        The shortest route.
    """
    
    # Initialize nodes.
    start_node = Node(start_pos, None)
    end_node = Node(end_pos, None)
    
    # Initialize open and closed lists.
    open_list = [start_node]
    closed_list = []
    
    # Start the looping through the open_list.
    while (len(open_list) > 0):
        # Find the best node, add to the closed_list.
        current_node, open_list = detach_least_f_node(open_list)
        closed_list.append(current_node)
        
        # Check if the current_node is the end_node.
        if current_node == end_node:
            return current_node.get_path()
        
        # Generate children nodes.
        children = generate_children(
            current_node, 
            end_node,
            closed_list,
            open_list,
            heatmap,
        )
        
        open_list.extend(children)

def generate_children(
    current_node: Node, 
    end_node: Node,
    closed_list: Sequence[Node],
    open_list: Sequence[Node],
    heatmap: np.ndarray,
) -> Sequence[Node]:
    """Find new possible nodes.
    
    Args:
        current_node: the current node.
        end_node: the end node.
        closed_list: exclude the child it's in the closed_list.
        open_list: exclude the child it's in the open_list.
        heatmap: the heatmap.
        
    Returns:
        A list of children: Sequence[Node].
    """
    children = []
    h_shape = heatmap.shape
    for vec in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
        new_pos = (
            current_node.pos[0] + vec[0],
            current_node.pos[1] + vec[1],
        )
        
        # Make sure it's not outside the map.
        if (
            new_pos[0] > (h_shape[0] - 1)
            or new_pos[1] > (h_shape[1] - 1)
            or new_pos[0] < 0
            or new_pos[1] < 0
        ):
            continue
            
        # Make sure the climb is 1 or lower.
        climb = heatmap[new_pos] - heatmap[current_node.pos]
        if climb > 1:
            continue
            
        child = Node(new_pos, current_node)
            
        # Exclude the child if it's already in the closed_list or open_list.
        if (child in closed_list) or (child in open_list):
            continue
            
        # Calculate f
        child.g = current_node.g + 1
        child.h = (
            abs(child.pos[0] - end_node.pos[0])
            + abs(child.pos[1] - end_node.pos[1])
        )
        child.f = child.g + child.h
        children.append(child)
    return children
    
    
def detach_least_f_node(node_list: Sequence[Node]):
    """Finds the node with the least f given a list of nodes.
    
    Args:
        node_list: a list of nodes.
        
    Returns:
        (
            a `Node` with the last f, 
            the remaining of the node_list
        )
    """
    best_idx = 0
    for idx, node in enumerate(node_list):
        if node.f < node_list[best_idx].f:
            best_idx = idx
    best_node = node_list.pop(best_idx)
    return best_node, node_list
    

@dataclasses.dataclass
class SolverA:
    """
    A solver instance.
    
    args:
        raw_data: the raw input data.
    """
    raw_data: str

        
    def find_answer(self) -> int:
        """Finds the answer.
        
        Returns:
            The answer.
        """
        path = search(*str_to_map(self.raw_data))
        return len(path)

In [ ]:
SolverA(test_data).find_answer()

In [ ]:
answer = SolverA(real_data).find_answer()
aocd.submit(answer, part="a", day=12, year=2022)

In [ ]:
from typing import Sequence, Union, Type, Tuple
import itertools

def str_to_map(string) -> np.ndarray:
    """Converts a block string to 2d np.ndarray.
    
    Args:
        string: the input string that represents the heat map.
        
    Returns:
        heatmap, start_position, end_position
    """
    
    lines = []
    start_position = end_position = None
    for ii, line in enumerate(string.splitlines()):
        row = []
        for jj, char in enumerate(line):
            if char == "S":
                row.append(0)
                start_position = (ii, jj)
            elif char == "E":
                row.append(25)
                end_position = (ii, jj)
            else:
                row.append(ord(char) - 97)
        lines.append(np.array(row))
    return np.array(lines), start_position, end_position
        
@dataclasses.dataclass
class Node:
    """Defines a node instance.
    
    Args:
        parent: the parent node of this node. 
        pos: a tuple of this node's position.
    """
    pos: Tuple[int, int]
    parent: Union[None, Type["Node"]]
        
    def __post_init__(self):
        self.f = 0
        self.g = 0
        self.h = 0
        
    def __eq__(self, compared: Type["Node"]):
        return self.pos == compared.pos
    
    def get_path(self):
        """Get the entire path for this node.
        
        Returns:
            The path positions: Sequence[Tuple[int, int]]
        """
        path = []
        this_node = self
        while this_node.parent is not None:
            path.append(this_node.pos)
            this_node = this_node.parent
        return path[::-1]
        
    
def inverse_search(
    heatmap: np.ndarray, 
    start_pos: Tuple[int, int], 
    end_pos: Tuple[int, int]
) -> Sequence[Tuple[int, int]]:
    """Uses A* to search.
    
    Args:
        the_map: the heatmap of terrain.
        start_pos: the starting position.
        end_pos: the end position.
    
    Return:
        The shortest route.
    """
    
    # Initialize nodes.
    start_node = Node(end_pos, None)
    end_node = Node(start_pos, None)
    
    # Initialize open and closed lists.
    open_list = [start_node]
    closed_list = []
    
    # Start the looping through the open_list.
    while (len(open_list) > 0):
        # Find the best node, add to the closed_list.
        current_node, open_list = detach_least_f_node(open_list)
        closed_list.append(current_node)
        
        # Check if the current_node is the end_node.
        if heatmap[current_node.pos] == 0:
            return current_node.get_path()
        
        # Generate children nodes.
        children = generate_inverse_children(
            current_node, 
            end_node,
            closed_list,
            open_list,
            heatmap,
        )
        
        open_list.extend(children)

def generate_inverse_children(
    current_node: Node, 
    end_node: Node,
    closed_list: Sequence[Node],
    open_list: Sequence[Node],
    heatmap: np.ndarray,
) -> Sequence[Node]:
    """Find new possible nodes.
    
    Args:
        current_node: the current node.
        end_node: the end node.
        closed_list: exclude the child it's in the closed_list.
        open_list: exclude the child it's in the open_list.
        heatmap: the heatmap.
        
    Returns:
        A list of children: Sequence[Node].
    """
    children = []
    h_shape = heatmap.shape
    for vec in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
        new_pos = (
            current_node.pos[0] + vec[0],
            current_node.pos[1] + vec[1],
        )
        
        # Make sure it's not outside the map.
        if (
            new_pos[0] > (h_shape[0] - 1)
            or new_pos[1] > (h_shape[1] - 1)
            or new_pos[0] < 0
            or new_pos[1] < 0
        ):
            continue
            
        # Make sure the drop is 1 or lower.
        drop = heatmap[current_node.pos] - heatmap[new_pos]
        if drop > 1:
            continue
            
        child = Node(new_pos, current_node)
            
        # Exclude the child if it's already in the closed_list or open_list.
        if (child in closed_list) or (child in open_list):
            continue
            
        # Calculate f
        child.g = current_node.g + 1
        # We don't know where the starting position should be.
        child.h = 0
        child.f = child.g + child.h
        children.append(child)
    return children
    
    
def detach_least_f_node(node_list: Sequence[Node]):
    """Finds the node with the least f given a list of nodes.
    
    Args:
        node_list: a list of nodes.
        
    Returns:
        (
            a `Node` with the last f, 
            the remaining of the node_list
        )
    """
    best_idx = 0
    for idx, node in enumerate(node_list):
        if node.f < node_list[best_idx].f:
            best_idx = idx
    best_node = node_list.pop(best_idx)
    return best_node, node_list
    

@dataclasses.dataclass
class SolverB:
    """
    A solver instance.
    
    args:
        raw_data: the raw input data.
    """
    raw_data: str

        
    def find_answer(self) -> int:
        """Finds the answer.
        
        Returns:
            The answer.
        """
        path = inverse_search(*str_to_map(self.raw_data))
        return len(path)

In [ ]:
SolverB(test_data).find_answer()

In [ ]:
answer = SolverA(real_data).find_answer()
aocd.submit(answer, part="b", day=12, year=2022)